## Data manpulation widgets toy examples
Currently to get this to work need to update the \_repr_html\_() method of the scip module to the following so that an optional scope can be passed in. This seemed like the nicest way to control the scope from within the widgets.

In [ ]:
def _repr_html_(input_scope=None):
    import inspect
    # Is there a better way to get the scope? The `7` is hard-coded for the
    # current IPython stack when calling _repr_html_ so this is bound to break.
    scope = input_scope if input_scope else inspect.stack()[7][0].f_globals
    from IPython import get_ipython
    ipython = get_ipython()
    out = ''
    for category in ['Variable', 'DataArray', 'Dataset']:
        names = ipython.magic(f"who_ls {category}")
        out += f"<details open=\"open\"><summary>{category}s:"\
               f"({len(names)})</summary>"
        for name in names:
            html = make_html(eval(name, scope))
            out += f"<details style=\"padding-left:2em\"><summary>"\
                   f"{name}</summary>{html}</details>"
        out += "</details>"
    from IPython.core.display import display, HTML
    display(HTML(out))

In [ ]:
from convert_helpers import ConvertWidget, DataCreationWidget, PlotWidget
import IPython.display as disp

### Data creation
Simple widget to create some pesudo real histogram data with sample, background and positional data. Currently only supports creating time of flight data.

In [ ]:
data_creation = DataCreationWidget(globals())
disp.display(data_creation)

### Data conversion

In [ ]:
data_conversion = ConvertWidget(globals())
disp.display(data_conversion)

### Data plotting
Plots the data aquired by evaluating the expression entered.

In [ ]:
plot_widget = PlotWidget(globals())
data_creation.subscribe(plot_widget)
data_conversion.subscribe(plot_widget)
disp.display(plot_widget)